For this project, you can scrape data for any specific product available on Amazon and analyze its customers’ reviews. After scraping, you can do sentiment analysis and perform the necessary statistical analysis to draw insightful conclusions.
1. Find the product 
2. requests the web content
3. web scraping 
4. Saving our data 
5. Clean the data
5. Analysis
6. conclusion 

## Library

In [112]:
import requests 
from bs4 import BeautifulSoup as soup

In [110]:
URL = 'https://www.amazon.com/STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
User_Agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
HEADERS = ({'User-Agent': User_Agent, 'Accept-Language': 'en-us,en;q=0.5'})

In [111]:
r = requests.get(URL, headers=HEADERS)
html_content = soup(r.content)


In [113]:
review_box = html_content.find_all(class_= "a-section review aok-relative", attrs = {'data-hook' : 'review'})


1. Date 
2. Rating 
3. Color
4. comment

In [114]:
import re
all_reviews = []
for review in review_box: 
    rev = []
    rating = review.find('span', class_= 'a-icon-alt').string[0]
    date_loc = review.find('span', attrs = {'data-hook':'review-date'}).string
    date = re.search('(?<=on)(.*)', date_loc).group(0)
    if(review.find('a', attrs={'data-hook': 'format-strip'})): 
        color = review.find('a', attrs={'data-hook': 'format-strip'}).string 
        color = re.search('(?<=:)(.*)', color).group(0)
    else: color = None
    if (review.find('span', attrs={'data-hook':'review-body'}).contents[1]):  
        comment = review.find('span', attrs={'data-hook':'review-body'}).contents[1].contents[0]
    else: 
        comment = None
    rev.extend([date, rating, color, comment])
    all_reviews.append(rev)
    

In [109]:
import pandas as pd 

df = pd.DataFrame(all_reviews, columns = ['Date', 'Rating', 'Color', 'Comment'])
df

,Date,Rating,Color,Comment
0,"February 9, 2023",5,None,I absolutely love these things. I got them bec...
1,"January 26, 2023",5,None,Sound: hands-down the sound is just like the e...
2,"January 30, 2023",5,None,I was wrong about these earbuds. At first I th...
3,"February 9, 2023",5,None,I picked these up to try them at work to help ...
4,"February 10, 2023",5,None,These would be perfect if there wasn't a sound...
5,"February 3, 2023",5,None,Looking for simple Bluetooth headphones to pai...
6,"February 15, 2023",4,None,I was really surprised at how great music soun...
7,"December 28, 2022",5,None,I have tested these earbuds in the last two we...
8,"February 5, 2023",4,None,I've had these for about a week now. They soun...
9,"January 14, 2023",5,None,I thought these would be good going by the rev...


## Request all the review pages 

In [129]:
def new_url(path):
    try:
        Base = 'https://www.amazon.com/'
        URL = Base+path['href']
        return URL
    except: 
        return None


In [130]:
def review_page(URL): 
    r = requests.get(URL, headers=HEADERS)
    html_content = soup(r.content)
    review_box = html_content.find_all(class_= "a-section review aok-relative", attrs = {'data-hook' : 'review'})
    
    outer_box = html_content.find(id = 'cm_cr-review_list')
    if outer_box.find('li', class_ = 'a-last') is None:
        URL = None
    else: 
        next_page = outer_box.find('li', class_ = 'a-last')
        path = next_page.find('a', href=True)
        URL = new_url(path)
    
    return review_box, URL
    

In [131]:
URL = 'https://www.amazon.com/STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
all_review_box = []
i = 0
while URL is not None: 
    review_box, URL = review_page(URL)
    all_review_box.append(review_box)
    i += 1
    print(f'{i}. {URL}')
    
    
    

1. https://www.amazon.com//STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber=2&reviewerType=all_reviews
2. https://www.amazon.com//STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_arp_d_paging_btm_3?ie=UTF8&pageNumber=3&reviewerType=all_reviews
3. https://www.amazon.com//STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_arp_d_paging_btm_4?ie=UTF8&pageNumber=4&reviewerType=all_reviews
4. https://www.amazon.com//STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_arp_d_paging_btm_5?ie=UTF8&pageNumber=5&reviewerType=all_reviews
5. https://www.amazon.com//STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_arp_d_paging_btm_6?ie=UTF8&pageNumber=6&reviewerType=all_reviews
6. https://www.amazon.com//STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_

47. https://www.amazon.com//STADOR-Bluetooth-Headphones-Waterproof-Earphones/product-reviews/B0BNBCXNP4/ref=cm_cr_arp_d_paging_btm_48?ie=UTF8&pageNumber=48&reviewerType=all_reviews


TypeError: 'NoneType' object is not subscriptable

In [132]:
all_review_box

[[<div class="a-section review aok-relative" data-hook="review" id="R3C4IOSYRF881P"><div class="a-row a-spacing-none" id="R3C4IOSYRF881P-review-card"><div class="a-section celwidget" id="customer_review-R3C4IOSYRF881P"><div class="a-row a-spacing-mini" data-hook="genome-widget"><a class="a-profile" data-a-size="small" href="/gp/profile/amzn1.account.AFK5GKUZEG6NTB7H54XXJLS5XCKA/ref=cm_cr_arp_d_gw_btm?ie=UTF8"><div aria-hidden="true" class="a-profile-avatar-wrapper"><div class="a-profile-avatar"><img class="a-lazy-loaded" data-src="https://images-na.ssl-images-amazon.com/images/S/amazon-avatars-global/default._CR0,0,1024,1024_SX48_.png" src="https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/grey-pixel.gif"/><noscript><img src="https://images-na.ssl-images-amazon.com/images/S/amazon-avatars-global/default._CR0,0,1024,1024_SX48_.png"/></noscript></div></div><div class="a-profile-content"><span class="a-profile-name">Mark in Los Angeles</span></div></a></div><div class="a

In [134]:
import re
all_reviews = []
for review_box in all_review_box:
    for review in review_box: 
        rev = []
        rating = review.find('span', class_= 'a-icon-alt').string[0]
        date_loc = review.find('span', attrs = {'data-hook':'review-date'}).string
        date = re.search('(?<=on)(.*)', date_loc).group(0)
        try:
            if (review.find('span', attrs={'data-hook':'review-body'}).contents[1]):  
                comment = review.find('span', attrs={'data-hook':'review-body'}).contents[1].contents[0]
            else: 
                comment = None
        except: 
            comment = None
        rev.extend([date, rating, comment])
        all_reviews.append(rev)
        

In [135]:
import pandas as pd 

df = pd.DataFrame(all_reviews, columns = ['Date', 'Rating', 'Comment'])
df

,Date,Rating,Comment
0,"February 9, 2023",5,I absolutely love these things. I got them bec...
1,"January 26, 2023",5,Sound: hands-down the sound is just like the e...
2,"January 30, 2023",5,I was wrong about these earbuds. At first I th...
3,"February 9, 2023",5,I picked these up to try them at work to help ...
4,"February 10, 2023",5,These would be perfect if there wasn't a sound...
...,...,...,...
465,"January 20, 2023",5,None
466,"November 14, 2022",5,Excellent sound quality and a great price!
467,"October 23, 2022",5,So easy to sync to iPhone. Came pre-charged so...
468,"November 18, 2022",5,Perfect headphones for the gym! Easy to connec...


In [142]:
df.Date = pd.to_datetime(df.Date)
df.Rating = df.Rating.astype('int')
df.Comment = df.Comment.str.strip()

In [140]:
df.dtypes

Date       datetime64[ns]
Rating              int64
Comment            object
dtype: object

In [144]:
df.to_csv('customerReviews.csv', index=False)

In [146]:
df = pd.read_csv('customerReviews.csv')

In [156]:
import datetime
df.Date = pd.to_datetime(df.Date)
df['Month'] = df['Date'].dt.month
df['Year'] = df.Date.dt.year
df.head(5)


,Date,Rating,Comment,Month,Year
0,2023-02-09,5,I absolutely love these things. I got them bec...,2,2023
1,2023-01-26,5,Sound: hands-down the sound is just like the e...,1,2023
2,2023-01-30,5,I was wrong about these earbuds. At first I th...,1,2023
3,2023-02-09,5,I picked these up to try them at work to help ...,2,2023
4,2023-02-10,5,These would be perfect if there wasn't a sound...,2,2023


In [160]:
df.Rating.value_counts()
df.Month.value_counts()


1     160
2     107
11    107
12     69
10     27
Name: Month, dtype: int64